In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pima-indians-diabetes-database/diabetes.csv


In [2]:
df = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [4]:
X = df.iloc[:,0:8].values
Y = df.iloc[:,-1].values

X.shape , Y.shape

((768, 8), (768,))

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X = scaler.fit_transform(X)
X.shape

(768, 8)

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=2)
X_train.shape , Y_train.shape

((614, 8), (614,))

In [30]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [8]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train,Y_train,batch_size=32,epochs=100,validation_data=(X_test,Y_test))

Epoch 1/100
20/20 [==============================] - 1s 16ms/step - loss: 0.6920 - accuracy: 0.5977 - val_loss: 0.6392 - val_accuracy: 0.6688
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6340 - accuracy: 0.6694 - val_loss: 0.5977 - val_accuracy: 0.7208
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5928 - accuracy: 0.7117 - val_loss: 0.5630 - val_accuracy: 0.7338
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5643 - accuracy: 0.7378 - val_loss: 0.5375 - val_accuracy: 0.7273
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5415 - accuracy: 0.7345 - val_loss: 0.5180 - val_accuracy: 0.7468
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5235 - accuracy: 0.7508 - val_loss: 0.5039 - val_accuracy: 0.7727
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5100 - accuracy: 0.7508 - val_loss: 0.4941 - val_accuracy: 0.7727
Epoch 8/100


### **KerasTuner Hyperparameter:**
#### **1. To Select Appropriate Optimizer**

In [9]:
!pip install -U keras-tuner

In [10]:
import keras_tuner as kt

In [11]:
def build_model(hp):
    
    model = Sequential()
    
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    
    optimizer = hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
    # hp.choice contain a string and list of value ,string contain any name but it nessary to assigne
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

In [12]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

In [13]:
tuner.search(X_train,Y_train,epochs=5,validation_data=(X_test,Y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.26623377203941345

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 09s


In [14]:
# Best perform model
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [15]:
# Use the Best model
model = tuner.get_best_models(num_models=1)[0]

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(X_train,Y_train,batch_size=32,epochs=100,initial_epoch=4,validation_data=(X_test,Y_test))

Epoch 5/100
20/20 [==============================] - 1s 16ms/step - loss: 0.5212 - accuracy: 0.7720 - val_loss: 0.5106 - val_accuracy: 0.7727
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5044 - accuracy: 0.7785 - val_loss: 0.4993 - val_accuracy: 0.7792
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4912 - accuracy: 0.7769 - val_loss: 0.4904 - val_accuracy: 0.7792
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4819 - accuracy: 0.7752 - val_loss: 0.4829 - val_accuracy: 0.7727
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4747 - accuracy: 0.7769 - val_loss: 0.4789 - val_accuracy: 0.7727
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4698 - accuracy: 0.7752 - val_loss: 0.4761 - val_accuracy: 0.7922
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4652 - accuracy: 0.7785 - val_loss: 0.4720 - val_accuracy: 0.7922
Epoch 12/1

#### **2. Number of nodes in Layers**

In [17]:
def build_model1(hp):
    
    model = Sequential()
    
    units = hp.Int('units',min_value=8,max_value=128)
    
    model.add(Dense(units=units ,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [18]:
tuner = kt.RandomSearch(build_model1,
                       objective='val_accuracy',
                       max_trials=5,
                       directory='mydir',
                       project_name='num_nodes')

In [19]:
tuner.search(X_train,Y_train,epochs=5,validation_data=(X_test,Y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7727272510528564

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 08s


In [20]:
tuner.get_best_hyperparameters()[0].values

{'units': 85}

In [21]:
# Used best parameter
model = tuner.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 85)                765       
                                                                 
 dense_1 (Dense)             (None, 1)                 86        
                                                                 
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(X_train,Y_train,batch_size=32,epochs=100,initial_epoch=4,validation_data=(X_test,Y_test))

Epoch 5/100
20/20 [==============================] - 1s 13ms/step - loss: 0.5070 - accuracy: 0.7590 - val_loss: 0.5022 - val_accuracy: 0.7727
Epoch 6/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4906 - accuracy: 0.7638 - val_loss: 0.4911 - val_accuracy: 0.7662
Epoch 7/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4808 - accuracy: 0.7704 - val_loss: 0.4813 - val_accuracy: 0.7662
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4718 - accuracy: 0.7720 - val_loss: 0.4766 - val_accuracy: 0.7792
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4664 - accuracy: 0.7752 - val_loss: 0.4767 - val_accuracy: 0.7727
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4616 - accuracy: 0.7720 - val_loss: 0.4737 - val_accuracy: 0.7792
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4587 - accuracy: 0.7801 - val_loss: 0.4718 - val_accuracy: 0.7727
Epoch 12/1

#### **3. To Select Number of Hidden_layers**

In [23]:
def build_model(hp):
    model = Sequential()
    
    model.add(Dense(72,activation='relu',input_dim=8))
    for i in range(hp.Int('num_layers',min_value=1,max_value=8)):
        model.add(Dense(72,activation='relu'))
    
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [24]:
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=3,
                       directory='mydir',
                       project_name='num_layer')

In [25]:
tuner.search(X_train,Y_train,epochs=5,validation_data=(X_test,Y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 07s


In [26]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4}

In [27]:
model = tuner.get_best_models(num_models=1)[0]

model.fit(X_train,Y_train,epochs=100,initial_epoch=4,validation_data=(X_test,Y_test))

Epoch 5/100
20/20 [==============================] - 1s 14ms/step - loss: 0.4529 - accuracy: 0.7769 - val_loss: 0.4632 - val_accuracy: 0.7597
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4397 - accuracy: 0.7883 - val_loss: 0.4633 - val_accuracy: 0.7857
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4328 - accuracy: 0.8013 - val_loss: 0.4844 - val_accuracy: 0.7468
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4245 - accuracy: 0.7964 - val_loss: 0.5061 - val_accuracy: 0.7468
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4148 - accuracy: 0.7997 - val_loss: 0.4882 - val_accuracy: 0.7662
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4199 - accuracy: 0.7948 - val_loss: 0.4857 - val_accuracy: 0.7338
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4005 - accuracy: 0.8078 - val_loss: 0.4917 - val_accuracy: 0.7338
Epoch 12/1

#### **4. All Changes in one model: (layer ,node ,Activation ,dropout ,optimizer)**

In [38]:
def build_model2(hp):
    
    model = Sequential()
    
    counter = 0
    
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        
        if counter == 0:  # First layer
            
            model.add(Dense(hp.Int('units' + str(i),min_value=8,max_value=128),# No.of nodes
                           activation= hp.Choice('activation' + str(i),values=['relu','tanh','sigmoid']),# Best activation
                           input_dim= 8))
            model.add(Dropout(hp.Choice('dropout' + str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))# no.of dropout nodes
            
        else: # Hidden layer
            
            model.add(Dense(hp.Int('units' + str(i),min_value=8,max_value=128),# No. of nodes
                           activation= hp.Choice('activation' + str(i),values=['relu','tanh','sigmoid'])))# Best activation
            model.add(Dropout(hp.Choice('dropout' + str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
            
        counter += 1
    model.add(Dense(1,activation='sigmoid'))  # Outer layer
    
    model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']),
                 loss= 'binary_crossentropy',
                 metrics=['accuracy'])
    
    return model

In [40]:
tuner = kt.RandomSearch(build_model2,
                       objective='val_accuracy',
                       max_trials=3,
                       directory='mydir',
                       project_name='final')

In [41]:
tuner.search(X_train,Y_train,epochs=5,validation_data=(X_test,Y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.7077922224998474

Best val_accuracy So Far: 0.7077922224998474
Total elapsed time: 00h 00m 15s


In [42]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 9,
 'units0': 10,
 'activation0': 'relu',
 'dropout0': 0.5,
 'optimizer': 'nadam',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1,
 'units3': 8,
 'activation3': 'relu',
 'dropout3': 0.1,
 'units4': 8,
 'activation4': 'relu',
 'dropout4': 0.1,
 'units5': 8,
 'activation5': 'relu',
 'dropout5': 0.1,
 'units6': 8,
 'activation6': 'relu',
 'dropout6': 0.1,
 'units7': 8,
 'activation7': 'relu',
 'dropout7': 0.1,
 'units8': 8,
 'activation8': 'relu',
 'dropout8': 0.1}

In [43]:
model = tuner.get_best_models(num_models=1)[0]

model.fit(X_train,Y_train,epochs=200,initial_epoch=4,validation_data=(X_test,Y_test))

Epoch 5/200
20/20 [==============================] - 5s 18ms/step - loss: 0.6863 - accuracy: 0.6368 - val_loss: 0.6768 - val_accuracy: 0.7078
Epoch 6/200
20/20 [==============================] - 0s 5ms/step - loss: 0.6806 - accuracy: 0.6352 - val_loss: 0.6645 - val_accuracy: 0.7078
Epoch 7/200
20/20 [==============================] - 0s 5ms/step - loss: 0.6729 - accuracy: 0.6368 - val_loss: 0.6535 - val_accuracy: 0.7078
Epoch 8/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6656 - accuracy: 0.6368 - val_loss: 0.6429 - val_accuracy: 0.7078
Epoch 9/200
20/20 [==============================] - 0s 5ms/step - loss: 0.6650 - accuracy: 0.6368 - val_loss: 0.6342 - val_accuracy: 0.7078
Epoch 10/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6621 - accuracy: 0.6368 - val_loss: 0.6297 - val_accuracy: 0.7078
Epoch 11/200
20/20 [==============================] - 0s 5ms/step - loss: 0.6545 - accuracy: 0.6384 - val_loss: 0.6213 - val_accuracy: 0.7078
Epoch 12/2